This example is an MB-MVPA code that uses part of the data (Tom et al., 2007).<br>
Tom's full dataset used 16 subjects, but in this example, only 2 subjects are used to save resource.

### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.</b>

Most of mb-mvpa are wrapping nilearn, tensorflow, Keras and etc., so warning can occur from that libraries.<br>
This page does not print warning because most of them are can be ignored.<br>
<b> You do not need to remove the warning when you are actually using it. </b>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from mbmvpa.preprocessing.bids import bids_preprocess
from mbmvpa.preprocessing.events import events_preprocess
from mbmvpa.data.loader import prepare_dataset
from mbmvpa.utils.example_utils import load_example_data

Data download from AWS S3, ~ 1GB.<br>
If you run this example more than once, the data will already be downloaded.<br>
If so, do not download it again.

In [3]:
root = load_example_data("tom")

data load success! (tom)


### Preprocessing bids data including fMRI images
The MB-MVPA liabrary need to BIDS format including preprocessed nii images*(nii or nii.gz)* and behavior data file*(.tsv)*.<br>
The fMRI images are usually located here<br>
**derivatives/fmriprep/**<br>
And the behavior data are located here<br>
**subject/session/run/func**

In [4]:
X, voxel_mask, layout, data_root = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=2, nthread=4)

bids preprocessing done!                          : 100%|██████████| 6/6 [00:19<00:00,  3.19s/it] 


<b>X</b> (numpy.array): subject-wise & run-wise BOLD time series data. shape : subject # x run # x timepoint # x voxel #.<br>
<b>voxel_mask</b> (nibabel.Nifti1Image): a nifti image for voxel-wise binary mask (ROI mask)

Our mbmvpa is directly inputted into the data and modulation functions to be used to increase user's freedom and increase stability.<br>
The following three functions must be completed, each of which means:<br>
1. If you use **hbayesDM**, you should change the columns name to suit it. -> preprocess
2. If you only want to use behavioral data under certain conditions, you must define a condition function. -> condition
3. Finally, since we are model-based fMRI, we have to calculate the modulation.<br>
   The important point is that the behavioral data that we already have are in the **"row"**, and parameters estimated by model are in the **"param_dict"**.<br>
   -> modulation

If you have parameters already calculated and can provide them in *.tsv*, we also allow that.<br>
"**individual_params_custom"** has the role.<br>
In that case, the dm_model, or deicison making model, is not required.

In [8]:
dm_model, df_events, signals, time_masks, _ = \
    events_preprocess(root,
                      individual_params_custom="../example_data/tom_example/derivatives/fmriprep/mvpa/individual_params.tsv")

hbayesdm doing (model: ra_prospect)..                      :  67%|██████▋   | 4/6 [00:03<00:01,  1.00it/s]INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl



Model  = ra_prospect
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 1
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 2
 # of (max) trials per subject  = 256

Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl


calculating modulation..                          :  83%|████████▎ | 5/6 [12:07<01:49, 109.44s/it]         

Index(['onset', 'duration', 'parametric loss', 'distance from indifference',
       'parametric gain', 'gain', 'loss', 'ptval', 'respnum', 'respcat',
       'responsetime', 'subjid', 'run', 'gamble', 'cert'],
      dtype='object')
['onset', 'duration', 'parametric loss', 'distance from indifference', 'parametric gain', 'gain', 'loss', 'PTval', 'respnum', 'respcat', 'response_time', 'subjID', 'run', 'gamble', 'cert']
************************************
**** Model fitting is complete! ****
************************************


events preproecssing done!                        : : 7it [12:08, 104.09s/it]                     


### Load data and shape check

Through the above process, **X(fMRI images)** and **Y(modulation)** dataset can be obtained.<br>
Their shape[0] is must be equal.

In [9]:
X, y, voxel_mask = prepare_dataset(data_root)

### Fitting MVPA models & Results

In [11]:
from mbmvpa.utils.coef2map import get_map
from time import process_time

In [12]:
from mbmvpa.models.regressor import penalized_linear_regression

s = time.time()
coefs = penalized_linear_regression(X, y,
                                    layout,
                                    lambda_param=2.0,
                                    batch_size=256,
                                    N=30,
                                    verbose=1)

task_name = "tom2007_penalized_linear"
result = get_map(coefs, voxel_mask, task_name=task_name, map_type="z", save_path=".", sigma=1)
e = time.time()
print(f"elapsed time: {(e-s) / 60:.2f} minutes")

INFO:root:[1/30] - val_mse: 0.1719
INFO:root:[2/30] - val_mse: 0.1622
INFO:root:[3/30] - val_mse: 0.1578
INFO:root:[4/30] - val_mse: 0.1664
INFO:root:[5/30] - val_mse: 0.1609
INFO:root:[6/30] - val_mse: 0.1595
INFO:root:[7/30] - val_mse: 0.1461
INFO:root:[8/30] - val_mse: 0.1612
INFO:root:[9/30] - val_mse: 0.1683
INFO:root:[10/30] - val_mse: 0.1583
INFO:root:[11/30] - val_mse: 0.1578
INFO:root:[12/30] - val_mse: 0.1604
INFO:root:[13/30] - val_mse: 0.1406
INFO:root:[14/30] - val_mse: 0.1574
INFO:root:[15/30] - val_mse: 0.1632
INFO:root:[16/30] - val_mse: 0.1467
INFO:root:[17/30] - val_mse: 0.1489
INFO:root:[18/30] - val_mse: 0.1516
INFO:root:[19/30] - val_mse: 0.1675
INFO:root:[20/30] - val_mse: 0.1688
INFO:root:[21/30] - val_mse: 0.1518
INFO:root:[22/30] - val_mse: 0.1624
INFO:root:[23/30] - val_mse: 0.1563
INFO:root:[24/30] - val_mse: 0.1625
INFO:root:[25/30] - val_mse: 0.1614


KeyboardInterrupt: 

In [ ]:
from mbmvpa.models.regressor import mlp_regression

s = time.time()
coefs = mlp_regression(X, y,
                       layout,
                       layer_dims=[1024, 1024],
                       activation="linear",
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)

task_name = "tom2007_mlp"
result = get_map(coefs, voxel_mask, task_name=task_name, map_type="z", save_path=".", sigma=1)
e = time.time()
print(f"elapsed time: {(e-s) / 60:.2f} minutes")

In [ ]:
from mbmvpa.models.regressor import elasticnet

s = time.time()
coefs = elasticnet(X, y,
                   layout,
                   n_jobs=16,
                   verbose=1,
                   max_lambda=1,
                   n_samples=5000)
task_name = 'tom2007_elasticnet'
result = get_map(coefs, voxel_mask, task_name=task_name, map_type="z", save_path=".", sigma=1)
e = time.time()
print(f"elapsed time: {(e-s) / 60:.2f} minutes")